In [71]:
import os

save_path = "../../data/nlp/policy_data.zip"
os.system(
    f'curl -o "{save_path}" https://github.com/chatgpt-kr/chatgpt-api-tutorial/raw/main/ch05/data.zip'
)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0


0

In [72]:
os.system(f"unzip {save_path} -d ../../data/nlp/policy_data/")

Archive:  ../../data/nlp/data.zip


  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of ../../data/nlp/data.zip or
        ../../data/nlp/data.zip.zip, and cannot find ../../data/nlp/data.zip.ZIP, period.


2304

In [38]:
import sys
import os

sys.path.append(os.path.abspath("../../data/API_KEY"))

In [39]:
import openai
import OpenAI_API_Key

API_KEY = OpenAI_API_Key.API_KEY

os.environ["OPENAI_API_KEY"] = API_KEY

openai.__version__

'1.51.2'

In [40]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from collections import Counter

# 다수의 문서 불러오기
* `DirectoryLoader()` 를 통해 특정 디렉토리 내의 모든 문서를 로딩할 수 있다

In [70]:
# glob 옵션을 이용해 어떤 확장자를 가진 파일을 로딩할지 결정
loader = DirectoryLoader(
    "../../data/nlp/policy_data", glob="*.txt", loader_cls=TextLoader
)
documents = loader.load()

len(documents)

57

# Chunking

In [49]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

texts = text_splitter.split_documents(documents)
len(texts)

64

In [50]:
# 어떤 문서가 분할 되었는지 확인
texts[0]

Document(metadata={'source': '../../data/nlp/policy_data/29.txt'}, page_content='정책제목: 청년과 어르신 주거공유(한지붕세대공감)\n서울시청 주택정책과에서는 해당 지역 대학생들의 주거 마련을 위해 할머니와 할아버지들이 남는 방을 학생에게 저렴한 가격에 빌려주는 정책을 운영하고 있습니다. 이 정책은 대학가나 청년들이 많이 사는 지역에서 진행되며, 어르신들은 시세보다 저렴한 값으로 방을 빌려주고, 학생들은 무보증금으로 학교 근처에 근접한 주거공간을 제공받습니다.\n\n어르신들은 주택을 소유한 60세 이상 어르신으로서 신청할 수 있으며, 대학생은 서울시 소재 대학(원) 재학생 및 휴학생으로 신청 자격을 갖추고 있어야 합니다. 학력이나 전공에 대한 제한은 없으며, 취업을 하지 않은 미취업자들이 참여할 수 있습니다.\n\n이 정책은 2023년 1월 1일부터 2023년 12월 31일까지 운영되며, 신청은 서울주거포털 사이트(https://housing.seoul.go.kr)에서 온라인으로 진행됩니다. 운영 기관은 주관기관과 동일하며, 자세한 신청 방법 및 제출서류는 공고문을 참조하시기 바랍니다.\n\n자세한 사항은 서울주거포털 사이트(https://housing.seoul.go.kr/)에서 확인하실 수 있습니다.')

In [51]:
source_lst = []
for i in range(0, len(texts)):
    source_lst.append(texts[i].metadata["source"])

element_counts = Counter(source_lst)
filtered_counts = {key: value for key, value in element_counts.items() if value >= 2}
print("2개 이상으로 분할된 문서 :", filtered_counts)
print("분할된 텍스트의 개수 :", len(documents) + len(filtered_counts))

2개 이상으로 분할된 문서 : {'../../data/nlp/policy_data/49.txt': 2, '../../data/nlp/policy_data/48.txt': 2, '../../data/nlp/policy_data/40.txt': 2, '../../data/nlp/policy_data/23.txt': 2, '../../data/nlp/policy_data/36.txt': 2, '../../data/nlp/policy_data/22.txt': 2, '../../data/nlp/policy_data/31.txt': 2}
분할된 텍스트의 개수 : 64


# VectorDB 구성

In [52]:
embedding = OpenAIEmbeddings()
vectordb = Chroma.from_documents(documents=texts, embedding=embedding)

vectordb

# Retriever 생성

In [53]:
retriever = vectordb.as_retriever()

In [54]:
query = "신혼 부부를 위한 정책을 보여줘"
docs = retriever.get_relevant_documents(query)

/var/folders/h0/f86w2r851zn7yj1w80yw932h0000gn/T/ipykernel_29974/3452358216.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)


In [55]:
len(docs)

4

In [56]:
docs[0]

Document(metadata={'source': '../../data/nlp/policy_data/23.txt'}, page_content='이 정책은 주거마련에 대한 부담을 완화하여 혼인수 감소와 출산기피 현상을 해결하고, 더 나은 주거환경을 제공하기 위해 서울시에서 운영하는 정책입니다. 대상 가구는 총 8,000가구로 제한되며, 지원 기간은 2023년 1월 1일부터 2023년 12월 31일까지입니다.\n\n이 정책에 참여하기 위해서는 다음의 신청자격을 충족해야 합니다. 먼저, 서울시민이거나 대출 후 1개월 이내에 서울로 전입 예정이어야 합니다. 또한 혼인신고일 기준으로 7년 이내의 신혼부부이거나 서울시 추천서 신청일로부터 6개월 이내에 결혼식 예정인 예비신혼부부여야 합니다. 부부의 합산 연소득은 9천 7백만원 이하여야 하며, 본인 및 배우자는 무주택자여야 합니다. 또한, 특정 주택 조건을 충족하는 주택의 임대차계약을 체결한 자에게 대출이 지원됩니다.\n\n이 정책은 서울주거포털(https://housing.seoul.go.kr)을 통해 온라인으로 신청할 수 있습니다. 필요한 제출서류로는 주민등록등본, 가족관계증명서, 혼인관계증명서, 그리고 임대차계약서가 있습니다.\n\n이 정책은 서울시청 주택정책과에서 운영되며, 자세한 사항은 해당 사이트(https://housing.seoul.go.kr)에서 확인하실 수 있습니다.')

In [57]:
# 문서의 출처 확인
for doc in docs:
    print(doc.metadata["source"])

../../data/nlp/policy_data/23.txt
../../data/nlp/policy_data/23.txt
../../data/nlp/policy_data/40.txt
../../data/nlp/policy_data/40.txt


각각의 문서가 명확하게 다른 주제에 대해 이야기한다면 overlap 을 더 줘도 괜찮을 것

In [58]:
docs[1]

Document(metadata={'source': '../../data/nlp/policy_data/23.txt'}, page_content='정책제목: 신혼부부 임차보증금 지원\n정부에서는 주거 관련 정책을 통해 부담을 완화하여 더 나은 주거환경을 제공하고자 합니다. 현재 주거마련에 대한 부담으로 인해 혼인수가 감소하고 출산기피 현상이 발생하고 있습니다. 따라서 주거비 부담을 완화하여 이러한 문제를 해결하고, 좋은 주거환경을 제공하고자 합니다.\n\n이 정책은 서울시청 주택정책과에서 주관하며, 주거 마련에 대한 부담을 완화하기 위한 내용을 포함하고 있습니다. 지원 대상은 관내 임차보증금 7억 이내의 주택 또는 주거용 오피스텔에 대해 해당하는 서울시민이나 서울로 전입 예정인 자입니다. 대출한도는 임차보증금의 90% 이내 또는 2억원 중 작은 금액이며, 대출금의 최대 연 3.6% 이차보전 및 최장 10년까지 지원됩니다.\n\n주택조건과 대출형식은 한국주택금융공사 보증 및 협약은행(국민, 하나, 신한) 대출, 그리고 서울시 이차보전이 적용됩니다. 이 정책은 2023년 1월 1일부터 2023년 12월 31일까지 운영되며, 신청자격은 1세부터 100세까지의 연령을 가진 사람들이 해당합니다.\n\n대출을 받기 위한 추가 요건으로는 혼인신고일 기준으로 7년 이내의 신혼부부이거나 서울시 추천서 신청일로부터 6개월 이내에 결혼식 예정인 예비신혼부부여야 합니다. 또한 부부합산 연소득이 9천 7백만원 이하이고 본인 및 배우자가 무주택자여야 합니다. 대출을 받을 주택은 특정 조건을 충족하는 주택의 임대차계약을 체결한 사람들을 대상으로 합니다.\n\n이 정책은 서울주거포털(https://housing.seoul.go.kr)에서 온라인으로 신청할 수 있습니다. 필요한 서류로는 주민등록등본, 가족관계증명서, 혼인관계증명서, 그리고 임대차계약서가 제출되어야 합니다.\n\n이 정책은 주거마련에 대한 부담을 완화하여 혼인수 감소와 출산기피 현상을 해결하고, 더 나은 주거환경을 제공하기 위

## K 개의 결과 반환
* 질문에 대해 답변이 들어있는 문서는 chunk 를 모두 고려했을 때 최대 갯수가 2개임을 위에서 확인

In [59]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

# Langchain 모델 구축
RetrievalQA.from_chain_type()을 이용해 llm 모델 기반의 QA 챗봇을 구축합니다. 이번 실습에서는 프롬프트를 따로 만들지 않고 `stuff`로 설정하여 기본 프롬프트를 이용한 QA 챗봇을 만들어 봅니다.

참고로 `stuff`로 설정했을 때 기본 프롬프트는 다음과 같습니다.
```
Use the following pieces of context to answer the users question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
{텍스트}

{질문}
```

`{텍스트}`에는 사용자의 질문으로부터 높은 유사도를 가진 텍스트가 들어가게 됩니다. `{질문}` 부분은 사용자의 질문이 들어가게 됩니다.

`return_source_documents`는 챗봇의 답변에 사용된 텍스트들의 출처를 표시할 것인지를 의미하게 됩니다.

In [60]:
# 기본적으로 주어지는 QA Prompt를 사용할 예정(stuff)
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model_name="gpt-4o-mini", temperature=0),
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
)

## 질의하기

In [61]:
input_text = "대출과 관련된 정책이 궁금합니다."
chatbot_response = qa_chain(input_text)
chatbot_response

/var/folders/h0/f86w2r851zn7yj1w80yw932h0000gn/T/ipykernel_29974/1171444085.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chatbot_response = qa_chain(input_text)


{'query': '대출과 관련된 정책이 궁금합니다.',
 'result': '서울시에서 운영하는 대출 관련 정책으로는 학자금대출 신용회복 지원사업과 주거 마련을 위한 대출 지원 정책이 있습니다.\n\n1. **학자금대출 신용회복 지원사업**: 이 정책은 학자금 대출로 인해 신용이 떨어진 청년층을 지원하는 프로그램입니다. 서울에 거주하는 19세부터 39세까지의 청년들이 대상이며, 신용유의자 등록을 해제하기 위해 분할상환약정 체결을 지원하고 초입금을 제공합니다. 신청은 서울청년포털을 통해 가능하며, 매월 심사 및 발표가 이루어집니다.\n\n2. **주거 마련 대출 지원 정책**: 이 정책은 신혼부부 및 예비신혼부부를 대상으로 주거 마련에 대한 부담을 완화하기 위한 대출 지원입니다. 서울시민이거나 대출 후 1개월 이내에 서울로 전입 예정이어야 하며, 혼인신고일 기준으로 7년 이내의 신혼부부 또는 결혼식 예정인 예비신혼부부가 대상입니다. 부부의 합산 연소득은 9천 7백만원 이하여야 하며, 무주택자여야 합니다. 신청은 서울주거포털을 통해 온라인으로 가능합니다.\n\n자세한 내용은 각 정책의 관련 사이트를 참고하시기 바랍니다.',
 'source_documents': [Document(metadata={'source': '../../data/nlp/policy_data/35.txt'}, page_content='정책내용: 서울시 학자금대출 신용회복 지원사업\n서울특별시 미래청년기획단이 주최하는 금융 정책으로, 학자금 대출로 인해 신용이 떨어져 어려움을 겪고 있는 청년층을 위한 신용회복 지원입니다. 이 정책은 분할상환약정 체결을 지원하고 초입금을 제공함으로써 신용유의자 등록을 해제하는 내용을 포함하고 있습니다. 추가적인 자부담 없이 약정을 체결할 수 있도록 합니다.\n\n지원 대상은 서울에 거주하며 학자금 대출로 인한 신용유의자인 19세부터 39세까지의 청년들이며, 약 200여명을 지원합니다. 2018년부터 2022년에 지원을 받은 사람은 2023년 지원 대상에서 제외됩

In [62]:
def get_chatbot_response(chatbot_response):
    print(chatbot_response["result"].strip())
    print("\n문서 출처:")
    for source in chatbot_response["source_documents"]:
        print(source.metadata["source"])

In [63]:
chatbot_response = qa_chain("신혼 부부의 신혼집 마련을 위한 정책이 있을까?")
get_chatbot_response(chatbot_response)

네, 신혼 부부의 신혼집 마련을 위한 정책이 있습니다. 서울시에서 운영하는 "신혼부부 임차보증금 지원" 정책이 그 예입니다. 이 정책은 주거마련에 대한 부담을 완화하고 더 나은 주거환경을 제공하기 위해 마련되었습니다. 지원 대상은 서울시민이거나 서울로 전입 예정인 신혼부부이며, 혼인신고일 기준으로 7년 이내의 신혼부부 또는 결혼식 예정인 예비신혼부부가 포함됩니다. 대출한도는 임차보증금의 90% 이내 또는 2억원 중 작은 금액이며, 지원 기간은 2023년 1월 1일부터 2023년 12월 31일까지입니다. 자세한 사항은 서울주거포털에서 확인하실 수 있습니다.

문서 출처:
../../data/nlp/policy_data/23.txt
../../data/nlp/policy_data/23.txt


In [64]:
chatbot_response = qa_chain("전세자금이 부족한 사람을 위한 정책 이름이 뭐야?")
get_chatbot_response(chatbot_response)

저는 그에 대한 정보를 가지고 있지 않습니다.

문서 출처:
../../data/nlp/policy_data/40.txt
../../data/nlp/policy_data/40.txt


In [65]:
chatbot_response = qa_chain("희망두베 청년텅장을 어떤걸 지원하니>")  # 약간의 오타 발생
get_chatbot_response(chatbot_response)

희망두배 청년통장은 저소득 근로 청년들의 자립을 지원하기 위해 다음과 같은 내용을 지원합니다:

1. 저축 목적에 따른 월 10만원 또는 15만원의 저축액 선택 (저축기간 2년 또는 3년)
2. 지원 금액은 본인이 적립한 금액의 100%로 지원됩니다. 
   - 10만원 저축 시: 
     - 2년형은 480만원 이상(이자 포함)
     - 3년형은 720만원 이상(이자 포함)
   - 15만원 저축 시: 
     - 2년형은 720만원 이상(이자 포함)
     - 3년형은 1,080만원 이상(이자 포함)

이 외에도 교육비, 주거비, 결혼자금, 창업운영자금 등을 위한 저축을 지원하는 내용이 포함되어 있습니다.

문서 출처:
../../data/nlp/policy_data/40.txt
../../data/nlp/policy_data/37.txt


In [75]:
import gradio as gr

# 인터페이스를 생성.
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="청년정책챗봇")  # 청년정책챗봇 레이블을 좌측 상단에 구성
    msg = gr.Textbox(label="질문해주세요!")  # 하단의 채팅창의 레이블
    clear = gr.Button("대화 초기화")  # 대화 초기화 버튼

    # 챗봇의 답변을 처리하는 함수
    def respond(message, chat_history):
        result = qa_chain(message)
        bot_message = result["result"]
        bot_message += " # sources :"

        # 답변의 출처를 표기
        for i, doc in enumerate(result["source_documents"]):
            bot_message += "[" + str(i + 1) + "] " + doc.metadata["source"] + " "

        # 채팅 기록에 사용자의 메시지와 봇의 응답을 추가.
        chat_history.append((message, bot_message))
        return "", chat_history

    # 사용자의 입력을 제출(submit)하면 respond 함수가 호출.
    msg.submit(respond, [msg, chatbot], [msg, chatbot])

    # '초기화' 버튼을 클릭하면 채팅 기록을 초기화.
    clear.click(lambda: None, None, chatbot, queue=False)

# 인터페이스 실행.
demo.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 3.50.2, however version 5.0.1 is available, please upgrade.
--------
Keyboard interruption in main thread... closing server.
